<a href="https://colab.research.google.com/github/duartejr/classificador_de_imagens/blob/main/classificador_extracao_caracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de folhas de videira

Grapevine Leaves Image Dataset - Kaggle


## Exercícios

In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
# Diretório onde estão armazenadas as imagens
images_dir = '/content/drive/MyDrive/Blue_Edtech_Ciencia_Dados/Grapevine_Leaves_Image_Dataset'

# Lista de sub diretórios 
sub_dirs = os.listdir(images_dir)

### Questão 1) Implemente uma rotina de carregamento das imagens 


In [ ]:
import os

def load_images(img_dir, folders):
    '''
    Rotina para carregamento das imagens:
    Args:
    img_dir (string) : Diretório onde as imagens estão armazenadas.
    folder (list) : Lista com o nome dos subdiretórios onde as imagens estão armazenadas. 

    Returns:
    X (np.array) : Array com as imagens carregadas
    y (np.array) : Array com a categoria de cada uma das imagens
    '''
    X = []
    y = []

    for folder in folders:
        images = glob(f'{img_dir}/{folder}/*.png')
        for image_file in images:
            image = Image.open(image_file)
            X.append(image)
            y.append(folder)

    return np.array(X), np.array(y)

### Questão 2) Faça a leitura das imagens e responda



Leitura das imagens e das labels de cada uma delas.

In [ ]:
X, y = load_images(images_dir, sub_dirs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


#### a) Qual a resolução das imagens deste dataset?

In [ ]:
width, height = X[0].size
n_channels = len(X[0].getbands())
mode = X[0].mode

print(f'Resolução da imagem (Largura x Altura x Canais de cores) = {width}px x {height}px x {n_channels}')
print(f'Modo de cores da imagem: {mode}')

Resolução da imagem (Largura x Altura x Canais de cores) = 511px x 511px x 4
Modo de cores da imagem: RGBA


#### b) Quantas imagens por classe?

In [ ]:
def print_categories(data):
    unique, counts = np.unique(data, return_counts=True)

    for category, count in zip(unique, counts):
        print(f'Categoria: {category} | Nº de elementos: {count}')

print_categories(y)

Categoria: Ak | Nº de elementos: 100
Categoria: Ala_Idris | Nº de elementos: 100
Categoria: Buzgulu | Nº de elementos: 100
Categoria: Dimnit | Nº de elementos: 100
Categoria: Nazli | Nº de elementos: 100


#### c) Este dataset já está estruturado em conjuntos treinamento/teste ou deve-se adotar alguma metodologia na modelagem?

O conjunto ainda não está dividio em conjuntos de treinamento e teste por isto será necessário implementar um método para realizar a separação deste dois grupos. A separação deve ser feita de forma que se garanta a proporcionalidade de classes existentes no grupo original. Um método que pode ser utilizado é o `train_test_split` do Scikit Learn. Para garantir a proporcionalidade das amostras pode-se utilizar o método `stratify` do `train_test_split`.

### Questão 3) Faça a preparação do dataset para extração de características com a CNN VGG-19. Qual o formato do input da rede? Verifique se há necessidade de transformação da imagem e, se sim, implemente esta transformação 


O input da VGG-19 têm dimensão 224x224. As imagens do problema têm resolução 512x512 e 4 canais de cores. Será portanto necessário reduzir ajustar as dimensões das imagens para que seja possível utilizar a VGG-19.

In [ ]:
def convert_images(images_list):
    '''
    Método para ajustar as imagens para as dimensões de entrada da VGG-19
    Args:
    images_list (np.Array) : Array de imagens a serem convertidas

    Return:
    images_converted (np.Array) : Array com as imagens ajustadas à configuração de entrada da VGG-19.
    '''
    images_converted = []

    for image in images_list:
        image = image.resize((224,224))     # Redimensiona imagem
        image = image.convert('RGB')        # Converte de RGBA para RGB
        image = np.asarray(image)           # Imagem para o formato np.Array
        images_converted.append(image)
    
    return np.array(images_converted)


Conversão das imagens para o padrão de entrada da VGG-19.

In [ ]:
X_converted = convert_images(X)

### Questão 4) Implemente o processo de extração de características utilizando a VGG-19 e a transformação deste espaço de característica de forma adequada 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import vgg19

def load_model():
    '''
    Método para carregar a VGG-19
    '''
    print("Carregando o modelo VGG19-ImageNet ...")
    model = vgg19.VGG19(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
    model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return model

def predict(images, model):
    '''
    Método para realizar a seleção de características.
    Args:
    images (np.Array) : Array com o dado das imagens
    model : Modelo que será utilizado para a seleção de características

    Return:
    np.Array : Array com os valores das características das imagens.
    '''
    prediction = np.array(model.predict(images))
    return np.reshape(prediction , ((prediction.shape[0], sum(prediction.shape[1:]))))

In [ ]:
model = load_model()                        # Inicializando o modelo
X_features = predict(X_converted, model)    # Seleção das características
print('Shape das features identificadas: ', X_features.shape)

Carregando o modelo VGG19-ImageNet ...
Shape das features identificadas:  (500, 4096)


### Questão 5) Utilize o espaço de características:

#### a) Se o conjunto não estiver particionado, escolha uma técnica e justifique sua decisão. Implemente esta rotina.

Para particionar os grupos de treino e teste utilizarei o `train_test_split`. O grupo de treino terá 70% das amostras e o de teste 30% que é uma das divisões mais comumente utilizadas. Para garantir a proporcionalidade das amostras utilizarei o argumento `stratify` do `train_test_split`. Ao passar uma lista de referência para este argumento os conjuntos de treino e teste serão separados manetendo-se a proporcionalidade original das classes. Ou seja, como nos dados originais cada uma das amostras têm a mesma quantidade de exemplos será garantido que nos conjuntos de treino e teste terei exemplos de cada uma das categorias na mesma proporcionalidade.

In [ ]:
def split_sets(X, y):
    '''
    Método para realizar a separação dos grupos de treino e teste
    '''
    return train_test_split(X, y, test_size=.3, random_state=101, stratify=y)

X_train, X_test, y_train, y_test = split_sets(X_features, y)
print('Categorias no conjunto de treino:\n')
print_categories(y_train)
print()
print('='*80)
print('\nCategorias no conjunto de teste:\n')
print_categories(y_test)

Categorias no conjunto de treino:

Categoria: Ak | Nº de elementos: 70
Categoria: Ala_Idris | Nº de elementos: 70
Categoria: Buzgulu | Nº de elementos: 70
Categoria: Dimnit | Nº de elementos: 70
Categoria: Nazli | Nº de elementos: 70


Categorias no conjunto de teste:

Categoria: Ak | Nº de elementos: 30
Categoria: Ala_Idris | Nº de elementos: 30
Categoria: Buzgulu | Nº de elementos: 30
Categoria: Dimnit | Nº de elementos: 30
Categoria: Nazli | Nº de elementos: 30


#### b) Utilize os dados com os classificadores Árvores de Decisão e Naive-Bayes. Obtenha a acurácia dos modelos e avalie os resultados.

Classificação utilizando a árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

y_predicit_DTC = DTC.predict(X_test)
print(classification_report(y_test, y_predicit_DTC))

              precision    recall  f1-score   support

          Ak       0.61      0.57      0.59        30
   Ala_Idris       0.33      0.50      0.39        30
     Buzgulu       0.44      0.37      0.40        30
      Dimnit       0.42      0.47      0.44        30
       Nazli       0.72      0.43      0.54        30

    accuracy                           0.47       150
   macro avg       0.50      0.47      0.47       150
weighted avg       0.50      0.47      0.47       150



Classificação utilizando o Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train, y_train)
y_predict_GNB = GNB.predict(X_test)

print(classification_report(y_test, y_predict_GNB))

              precision    recall  f1-score   support

          Ak       0.79      0.37      0.50        30
   Ala_Idris       0.38      0.40      0.39        30
     Buzgulu       0.54      0.63      0.58        30
      Dimnit       0.37      0.43      0.40        30
       Nazli       0.65      0.73      0.69        30

    accuracy                           0.51       150
   macro avg       0.54      0.51      0.51       150
weighted avg       0.54      0.51      0.51       150



O modelo de Naive Bayes teve uma acurácia melhor que o da Árvore de Decisão, 0,54 contra 0,50. Porém, em ambos os casos, este valor é ainda é baixo, seria interessante tentar modificar os hiperparâmetros destes modelos para tentar melhorar a acurácia dos mesmos. Observa-se que as classes Ak e Nazli são as de melhor capacidade de predição em ambos os modeloos. E as classes Ala_Idris e Dmnit as que os modelos têm maior dificuldade de identifcar.  